In [36]:
from sklearn.externals import joblib
import pickle

In [37]:
from pydub import AudioSegment
import numpy as np
from scipy.io.wavfile import read
from scipy.fftpack import fft
import pydub
from scipy import signal
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import speech_recognition as sr

In [38]:
def dataSetAudios(nombre):
    lista = []
    listafin = []
    for a in range(1,7):
        audio = AudioSegment.from_file(f'../audios/{nombre}{a}.m4a', format='mp4')
        lista.append(audio)
    for a in lista:
        array = a.get_array_of_samples()[:528320]
        four = abs(fft(array))
        listafin.append(four)
    
    return listafin        

In [39]:
def convInt(x):
    x = int(x)
    return x

In [40]:
af = "Afecto"
ale = "Alegria"
enf = "Enfado"
trs = "Tristeza"
afecto = dataSetAudios(af)
alegria = dataSetAudios(ale)
enfado = dataSetAudios(enf)
tristeza = dataSetAudios(trs)

In [41]:
alegriad = {"transformada": [a for a in alegria], "etiqueta": 1}
enfadod = {"transformada": [a for a in enfado], "etiqueta": 2}
tristezad = {"transformada": [a for a in tristeza], "etiqueta":3}
afectod = {"transformada": [a for a in afecto], "etiqueta": 4}

In [42]:
todos = [alegriad,enfadod,tristezad,afectod]

In [43]:
def haceDf(elemento):
    temp = []
    for a in elemento:
        df = pd.DataFrame(elemento)
        temp.append(df)
    return pd.concat(temp).reset_index(drop=True)

In [44]:
al = pd.DataFrame(alegriad)
tr= pd.DataFrame(tristezad)
en = pd.DataFrame(enfadod)
af = pd.DataFrame(afectod)
todos = pd.concat([al,tr,en,af]).reset_index(drop=True)

In [45]:
X=np.concatenate((np.vstack(al.transformada),np.vstack(tr.transformada),np.vstack(en.transformada),np.vstack(af.transformada)))
y=np.concatenate((al.etiqueta,tr.etiqueta,en.etiqueta,af.etiqueta))

In [46]:
X.shape

(24, 528320)

In [47]:
y.shape

(24,)

In [48]:
X.shape

(24, 528320)

In [49]:
rfc = RandomForestClassifier(n_estimators = 200)
gbc = GradientBoostingClassifier()

In [50]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1)

In [51]:
gbc.fit(X_train, y_train)
y_pred = gbc.predict(X_test)

In [52]:
#joblib.dump(gbc, '../modelos/model_gbcx.pkl')

In [53]:
rfc.fit(X_train, y_train)
h_pred = rfc.predict(X_test)

In [54]:
#joblib.dump(rfc, '../modelos/model_rfcx.pkl')

In [55]:
accrandomf = accuracy_score(y_test, y_pred)
accgradient = accuracy_score(y_test,h_pred)
print(accrandomf, accgradient)

1.0 1.0


In [29]:
def procesaUnAudio(audio):
    lista = []
    array = audio.get_array_of_samples()[:528320]
    print(len(array))
    four = abs(fft(array))
    lista.append(four)
    return np.vstack(lista)

In [32]:
r = sr.Recognizer()

with sr.Microphone() as source:
    print("Expresa algo")
    audio = r.record(source, duration=13)
    print("Grabación terminada. Procesando...")


with open("audio.wav", "wb") as f:
        f.write(audio.get_wav_data())

audio = AudioSegment.from_file(f'audio.wav', format='mp4')
prueba = procesaUnAudio(audio)
predigo = gbc.predict(prueba)
predigo